# Travel Recommender

Discover your perfect vacation spot with our AI-powered chat application! Using Genui, our app analyzes user preferences and provides personalized travel recommendations for stunning destinations worldwide. Whether you're seeking a relaxing beach getaway, an adventurous mountain escape, or a cultural city tour, our intelligent chatbot suggests the best locations tailored to your interests. Simply chat, explore, and plan your next trip effortlessly. Let AI take the guesswork out of travel planning and find your ideal vacation in seconds!

## Business Goal:

The business goals for your AI-powered travel recommendation chat application could include:

User Engagement & Retention – Create an intuitive and interactive experience that keeps users returning for personalized travel suggestions.

Monetization – Generate revenue through affiliate partnerships with travel agencies, airlines, hotels, and tour operators.

Market Expansion – Grow the user base by targeting travel enthusiasts, digital nomads, and vacation planners worldwide.

Data Insights & Optimization – Leverage user interactions to refine AI recommendations and enhance customer satisfaction.


Scalability – Expand features over time, such as budget-based recommendations, itinerary planning, or real-time travel updates.

## ConsolidatedTravel
The "ConsolidatedTravel" dataset serves as the foundation for the travel recommender. It includes key details such as city names, best time to visit, descriptions, and hotel costs, all summarized in a single column. This sheet acts as the primary data source for the recommendation process.

## This notebook includes the following functionalities:


#### Imports Libraries:

    Uses pandas for data handling.

    IPython.display for displaying output.

    OpenAI API for generating recommendations.

    ReportLab for generating PDF reports.

    Tenacity for handling API request retries.

#### Loads Data:

    Reads a dataset from "ConsolidatedTravel.xlsx", which likely contains travel destination details.

#### Integrates OpenAI API:

    Reads an API key from "OPENAI_API_Key.txt".

    Stores the key in the environment for making requests to generate travel recommendations.

#### Processes Travel Data:

    Extracts summaries of destinations.

    Likely uses AI to refine suggestions based on user preferences.


## Import the libraries:

In [1]:
import pandas as pd
from IPython.display import display, HTML
# Import the libraries
import os, json, ast
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import textwrap
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import textwrap

# Set the display width to control the output width
pd.set_option('display.width', 100)


## Travel Data Processing
    Reads "ConsolidatedTravel.xlsx", which contains travel destinations, budget, best travel time, and descriptions.
    
    Extracts summaries from the dataset (df['Summary']), which are referenced when interacting with the user.

In [2]:
# Read the dataset and read the Laptop Dataset
df = pd.read_excel('ConsolidatedTravel.xlsx')
df

,City,Best TimeToVisit,City_desc,CountofPlaces,CostofStay,Summary
0,Manali,October-June,[' One of the most popular hill stations in Hi...,50,Hotel : 500 -5000 GuestHouse : 2000 -8000 ...,City : Manali ;; Best TimeToVisit : October-...
1,Leh Ladakh,July-October,"["" Ladakh is a union territory in the Kashmir ...",52,Hotel : 1000 - 4000 GuestHouse : 800 - 3000 ...,City : Leh Ladakh ;; Best TimeToVisit : July...
2,Coorg,September-June,[' Located amidst imposing mountains in Karnat...,34,Hotel : 2000 -8000 GuestHouse : 1000 - 4000...,City : Coorg ;; Best TimeToVisit : September...
3,Andaman,October-March,[' Replete with turquoise blue water beaches a...,52,Hotel : 500 - 10000 GuestHouse : 500 - 6000 ...,City : Andaman ;; Best TimeToVisit : October...
4,Lakshadweep,September-February,"["" Formerly known as Laccadive Islands, Laksha...",15,Hotel : 5000 -20000 GuestHouse : 3000 - 2000...,City : Lakshadweep ;; Best TimeToVisit : Sep...
...,...,...,...,...,...,...
95,Pushkar,October-March,[' Pushka is a small temple town located aroun...,20,Hotel : 2000 -8000 GuestHouse : 1000 - 4000 ...,City : Pushkar ;; Best TimeToVisit : October...
96,Chittorgarh,October-March,"[' Located in South-Eastern Rajasthan, Chittor...",19,Hotel : 2000 -8000 GuestHouse : 1000 - 4000 ...,City : Chittorgarh ;; Best TimeToVisit : Oct...
97,Nahan,September-November,"[' Away from the buzzing crowd of the city, Na...",20,Hotel : 2000 -8000 GuestHouse : 1000 - 4000 ...,City : Nahan ;; Best TimeToVisit : September...
98,Lavasa,October-March,"["" Known as India's newest hill station, the L...",5,Hotel : 500 - 6000 GuestHouse : 800 - 3000 ...,City : Lavasa ;; Best TimeToVisit : October-...


In [3]:
# Read the OpenAI API key
openai.api_key = open("OPENAI_API_Key.txt", "r").read().strip()
os.environ['OPENAI_API_KEY'] = openai.api_key

In [4]:
df['Summary'][0]

'City : Manali  ;;  Best TimeToVisit : October-June  ;;  City_desc : [\' One of the most popular hill stations in Himachal, Manali offers the most magnificent views of the Pir Panjal and the Dhauladhar ranges covered with snow for most parts of the year. \', \'\', " With the Covid-19 pandemic going on, Manali has evolved to be a place loved by young people looking for longer stays for workation. With ambient cafes, good wifi availability, small eateries and convenient shops, Old Manali is among the favorite neighbourhood for such people. There are many homestays and hostels offering dorm beds for cheap for longer durations. Manali has many trekking options around it, making it a great base for exploring this side of Himalayas. River Beas provides great rafting options in the nearby town of Kullu. Adjoining Parvati river, lies the Parvati Valley with  Kasol, Manikaran, Tosh and small villages attracting travellers for longer stays. Atal Tunnel now allows travellers to reach Sissu within

## User Interaction & Data Collection
### initialize_conversation():

#### Starts a conversation with the user.

    Asks structured questions about their preferences (e.g., destination type, activities, budget, climate, travel season, and group size).
    
    Ensures all key details are collected before making a recommendation.
    
    Uses a dictionary to store user responses.
    
    Example questions asked:
    
    "What type of location do you prefer? (Beach, Hills, Forest, Desert, Island)"
    
    "What kind of activities are you interested in? (Adventure, Culture, Relaxation, Sightseeing)"
    
    "What is your approximate budget for this trip?"
    
    "When are you planning to travel?"

### Implementing Intent Clarity and Intent Confirmation Layers


In [5]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"

    example_user_dict = {'Location': "Hills",
                        'Activities':"Trekking",
                        'TimeOfYear': "Jan-Mar",
                        'Climate': "Cold",
                        'Group': "couple",
                        'RoomBudget': "2000"}

    example_user_req = {'Location': "_",
                        'Activities': "_",
                        'TimeofYear': "_",
                        'Climate': "_",
                        'Group': "_",
                        'RoomBudget': "_"}

    system_message = f"""
    You are a intelligent travel advisor and recommend the best travel destination based on user preferences. 
    You need to ask relevant questions and understand the user profile by analysing the user's responses.

    Begin with asking his name. This is very important and you will be addressing him on this

    To make an informed suggestion, ask the user for the following key details
    {delimiter}
    Preferred Destination Type – Do they prefer beaches, hills, forests, islands, or deserts?
    Activities of Interest – Are they looking for adventure (trekking, water sports), cultural experiences (heritage sites, temples), or relaxation (resorts, spas)?
    Budget – What type of room are they looking for {delimiter} present the room details along with budget in amount (numbers) for them to choose). 
    You will be penalized if you do not do this{delimiter}
    
    Climate Preference – Do they prefer cold, moderate, or tropical weather? {delimiter} present examples for clear response {delimiter}
    Travel Timeframe – What is their preferred season of travel (season or specific months)?
    Group Type – Are they traveling solo, as a couple, with family, or in a group?
    Cultural Interests – Do they prefer places rich in history, local culture, and traditions?

    {delimiter}

    VERY IMPORTANT. If the value for any key is '_' please ensure to ask the question. You have to capture all the values before recommending to the user. 
    {delimiter}

    For each of the question, please few examples so that you can guide the  user for better options. Otherwise you will be penalized

    The python dictionary looks like this
    {{'Destination': 'values','Activities': 'values','TimeofYear': 'values','Climate': 'values','Group': 'values','RoomBudget': 'values'}}
    The value for 'Budget' should be a numerical value extracted from the user's response.
    
    {delimiter}

    Based on their answers, recommend the best destination from the available options, providing reasons why it fits their needs. 
    Also, suggest must-visit attractions and activities they can explore during their trip.
    
    {delimiter}

    Please reask the question in a differnet way if any of the response is inaccurate or not as expectation (IMPORTANT)

    {delimiter}

    Key Values that you need to capture based on the discussion with user:
    Location – ["The specific location recommended"]
    Activities –  ["Adventure, cultural experiences, relaxation, sightseeing, etc"]
    TimeofYear – ["During whuch months is the traveller planning"]
    Climate – ["The Climate the user wants to explore"]
    Group - ["Whether the user is planning to travel Solo, Couple or Family etc]
    RoomBudget - ["The budget they have for room"]



    - Do not randomly assign values to any of the keys.
    - The values need to be inferred from the user's response.

    {delimiter}

    Please ask the question again if any of the 6 keys ('Location','Activities','TimeofYear','Climate','Group','RoomBudget') does not have a value. It cannot be "_" or spaces etc.
    It has to be a valid values.    otherwise you will be penalized
    
    {delimiter}

    To fill the dictionary, you need to have the following chain of thoughts:
    Follow the chain-of-thoughts below and only output the final updated python dictionary for the keys as described in {example_user_req}. \n


    Thought 2: Now, you are trying to fill the values for the rest of the keys which you couldn't in the previous step.
    Remember the instructions around the values for the different keys.
    Ask questions you might have for all the keys to strengthen your understanding of the user's profile.
    If yes, move to the next Thought. If no, ask question on the keys whose values you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the key you want to understand value for.
    {delimiter}

    {delimiter}
    Thought 3: Check if you have correctly updated the values for the different keys in the python dictionary.
    If you are not confident about any of the values, ask clarifying questions.
    {delimiter}


    Here is a sample conversation between the user and assistant:

    Assistant: " Hello! I’d love to help you find the perfect travel destination. To personalize my recommendation, I need a few details from you.
    
                What type of location do you prefer? (Beach, Hills, Forest, Desert, Island)
                What kind of activities are you interested in? (Adventure, Culture, Relaxation, Sightseeing)
                What is your approximate budget for this trip?
                Do you have any climate preferences? (Cold, Warm, Tropical, Moderate)
                When are you planning to travel? (Specific months or seasons)
                Are you traveling solo, as a couple, with family, or with friends?
                
    Let me know, and I’ll suggest the best travel destination for you!  "



    User: "I’m looking for a beach destination with a mix of adventure and relaxation. My budget is around $1000. 
I prefer a tropical climate and plan to travel in December."

Remember you need not NOT ask all questions at once and go with 1-2 questions initially and then based on that ask further questions. You will be penalized if 
you do not follow this.


List of all travel locations along with budget, time of visit, description of city along with experience can be found here. 
{delimiter} {

df['Summary']}   

{delimiter} 
Please refer this also when you are interacting with the user


    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message}]
    # conversation = system_message
    return conversation

## Recommendation Logic
    Uses AI to match the user’s inputs with a suitable destination.
    
    References the travel dataset to provide:
    
    Recommended destination.
    
    Reasoning behind the suggestion.
    
    Must-visit attractions and activities.
    
    Ensures:
    
    All required user details are gathered before recommending.
    
    Follows a structured chain-of-thought process:
    
    Initial Questioning – Collect user preferences.
    
    Clarification – Asks follow-up questions if needed.
    
    Finalization – Ensures all details are valid before making a recommendation.



### `get_chat_completions()`:

This function perform LLM call using the Chat Completions API to get the LLM response.

In [6]:
# Define a Chat Completions API call
# Retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_chat_completions(input, json_format = False):
    MODEL = 'gpt-3.5-turbo'

    system_message_json_output = """<<. Return output in JSON format to the key output.>>"""

    # If the output is required to be in JSON format
    if json_format == True:
        # Append the input prompt to include JSON response as specified by OpenAI
        input[0]['content'] += system_message_json_output
        # JSON return type specified
        chat_completion_json = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            response_format = { "type": "json_object"},
            seed = 1234)

        output = json.loads(chat_completion_json.choices[0].message.content)

    # No JSON return type specified
    else:
        chat_completion = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            seed = 2345)

        output = chat_completion.choices[0].message.content

    return output

### Define a function called moderation_check that takes user_input as a parameter.


In [7]:

def moderation_check(user_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = openai.moderations.create(input=user_input)

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if response.results[0].flagged == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

### `intent_confirmation_layer()`:


In [8]:
def intent_confirmation_layer(response_assistant):

    delimiter = "####"

    prompt = f"""
    You are a senior evaluator who has an eye for detail.The input text will contain a user requirement captured through 6 keys.
    You are provided an input. You need to evaluate if the input text has the following keys:



                       {{'Location': 'values',
                        'Activities': 'values',
                        'TimeofYear': 'values',
                        'Climate': 'values',
                        'Group': 'values',
                        'RoomBudget': 'values'}}
                        
    The 'RoomBudget' key can take only a numerical value.
    Next you need to evaluate if the keys have the the values filled correctly.

    If any of the values are '_' please outout 'No', else you must output 'Yes'. You will be penalized if you do not follow this.
    
    Thought 1 - Output a string 'Yes' if the values are correctly filled for all keys, otherwise output 'No'.
    THought 2 - Think carefully before the answering.


    Only output a one-word string in JSON format at the key 'result' - Yes/No.


    Please check if all the 6 keys ('Location','Activities','TimeofYear','Climate','Group','RoomBudget') has a  value before responding, 
    otherwise you will be penalized. Your response will only be a 'Yes' or 'No'
    


    Also none of the keys should have a value of "_" or spaces. You need to check all of this before confirming.
    
        """
    messages=[{"role": "system", "content":prompt },
              {"role": "user", "content":f"""Here is the input: {response_assistant}""" }]

    response = openai.chat.completions.create(
                                    model="gpt-3.5-turbo",
                                    messages = messages,
                                    response_format={ "type": "json_object" },
                                    seed = 1234
                                    # n = 5
                                    )

    json_output = json.loads(response.choices[0].message.content)

    return json_output

### `dictionary_present()`:

This function checks if the final understanding of user's profile is returned by the chatbot is a Python dictionary or not. This is important as it'll be used later on for finding the right laptops using dictionary matching.

In [9]:
def dictionary_present(response):
    delimiter = "####"

    user_req=          {'Location': ['hills'],
                        'Activities': ['trekking','swimming'],
                        'TimeofYear': ['January','February'],
                        'Climate': ['Winter','spring'],
                        'Group': ['family','couple'],
                        'RoomBudget': [2000,3000]}

    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract the relevant values from the input and return only the python dictionary in JSON format.
            The output should match the format as {user_req}.

            {delimiter}
            Make sure that the value of budget is also present in the user input. ###
            The output should contain the exact keys and values as present in the input.
            Ensure the keys and values are in the given format:


            {{'Location': ['hills'],
                        'Activities': ['trekking','swimming'],
                        'TimeofYear': ['January','February'],
                        'Climate': ['Winter'],
                        'Group': ['family'],
                        'RoomBudget': [2000]}}


            {delimiter}

            """
    messages = [{"role": "system", "content":prompt },
                {"role": "user", "content":f"""Here is the user input: {response}""" }]

    confirmation = get_chat_completions(messages, json_format = True)

    return confirmation

## Top_Three_Suggestions (Analyse and Provide the user top 3 recommendations)

In [10]:
def Top_Three_Suggestions(conversation,recommendations):

    delimiter = "####"
    tabbled = '  '

    prompt = f"""
    
    You are a professional travel agent. Your task is to create a personalized **travel proposal** 
    recommending the **top 3 best destinations** based on the user's preferences in {recommendations}. 
    The destinations should be ranked from **most recommended to least recommended** based on their fit with the user's needs.  

You need to provide a detailed report that can be presented to the Customer. Review the entre Chat along with {recommendations} when you prepare the content.

#Return the DETAILED proposal in structured text format so it can be formatted into a **PDF** (Atleast 300 words)

Details of the Place, Activities, Budget, Climate should all be present. Please have a customized message to the user and address him by his name

YOU WILL BE PENALIZED IF YOU DO NOT DO THIS
    
           
    
    """
    
    # messages=[{"role": "system", "content":conversation },
    #           {"role": "user", "content":f"""Here is the input: {prompt}""" }]

    conversation.append({"role": "user", "content": str({prompt})})

#    print (conversation)
#    print (messages)
#    print ("message above")
    response = openai.chat.completions.create(
                                    model="gpt-3.5-turbo",
                                    messages = conversation,
 #                                   response_format={ "type": "json_object" },
                                    seed = 1234
                                    # n = 5
                                    )
#    print ("in output")
   # output = response.choices[0].message.content

    return response
    # json_output = json.loads(response.choices[0].message.content)

    # return json_output

### Dialogue Management System

Bringing everything together, we create a `diagloue_mgmt_system()` function that contains the logic of how the different layers would interact with each other. This will be the function that we'll call to initiate the chatbot

In [11]:
def dialogue_mgmt_system():
    
    global user_criteria_built
    global recommendations
    global conversation
    
    conversation = initialize_conversation()

    introduction = get_chat_completions(conversation)

    conversation.append({"role": "assistant", "content": introduction})

    display(introduction + '\n')


    user_input = ''

    while(user_input != "exit"  and user_criteria_built=="N"):

        user_input = input("")

        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break

        if user_criteria_built == "N":

            conversation.append({"role": "user", "content": user_input})

            response_assistant = get_chat_completions(conversation)
            
            moderation = moderation_check(response_assistant)
            if moderation == 'Flagged':
                display("Sorry, this message has been flagged. Please restart your conversation.")
                break


            confirmation = intent_confirmation_layer(response_assistant)

 #           print("Intent Confirmation Yes/No:",confirmation.get('result'))

            if "No" in confirmation.get('result'):
                conversation.append({"role": "assistant", "content": str(response_assistant)})
                print("\n" + str(response_assistant) + "\n")

            else:
                #print("\n" + str(response_assistant) + "\n")
                print('\n' + "Thank you for providing the details. Please hold while we advise on top 3 best locations!" + '\n')
                conversation.append({"role": "assistant", "content": str(response_assistant)})

                user_criteria_built = "Y"
                recommendations = dictionary_present(response_assistant)
                
                print("\n" + str(recommendations) + "\n")
#                print ("coming out of loop")


    
    # with open("output.txt", "w", encoding="utf-8") as file:
    #     file.write(output_recommendations)

## PDF Report Generation
Uses ReportLab to generate a PDF report summarizing the recommended destination, activities, and budget.

In [12]:


user_criteria_built = "N"
recommendations = ""
conversation =[]

pdf_filename = "TravelRecommendation.pdf"
dialogue_mgmt_system()
output_recommendations = ""


if (user_criteria_built =="Y"):
#    print ("going to fetch top 3 locations")
    output_recommendations = Top_Three_Suggestions(conversation,recommendations)
#    print (output_recommendations)


    
    x, y = 25, 750 
    
    c = canvas.Canvas(pdf_filename, pagesize=letter)
    c.setFont("Helvetica", 8)
    
    for line in output_recommendations.choices[0].message.content.splitlines():
        if (line == "") or (line == "\n"):
            continue
        else:
    
            if len(line)> 100:
                wrapped_lines = textwrap.wrap(line, width=150)
                for part in wrapped_lines:
                    c.drawString(x, y, part.strip())
                    y -= 20
    
            else:
                c.drawString(x, y, line.strip())
                y -= 20  # Move down for the next line
    
    c.save()

    print (" Please check the PDF file TravelRecommendation.pdf that has details on the top 3 locations we would advise. Thank you and looking forward to serve you")

'Hello! I’d love to help you find the perfect travel destination. To personalize my recommendation, I need a few details from you.\n\nWhat type of location do you prefer? (Beach, Hills, Forest, Desert, Island)\n'

 Forest



Great choice! 

What kind of activities are you interested in? Are you looking for adventure, cultural experiences, relaxation, or sightseeing?



 Adventure



Awesome! 

What budget range are you considering for your trip? Some options include:
1. Budget rooms ($50-$100 per night)
2. Standard rooms ($100-$200 per night)
3. Luxury rooms ($200+ per night) 

Please select the budget range that suits you best.



...


Could you please specify the month or season for your summer travel plans?



 summer



Thank you for confirming!

Are you traveling solo, as a couple, with family, or in a group?



 couple



Thank you for providing the details. Please hold while we advise on top 3 best locations!


{'Location': ['Forest'], 'Activities': ['Adventure'], 'TimeofYear': ['Summer'], 'Climate': ['Cold'], 'Group': ['Couple'], 'RoomBudget': [100]}

going to fetch top 3 locations
 Please check the PDF file TravelRecommendation.pdf that has details on the top 3 locations we would advise. Thank you and looking forward to serve you
